## Generazione numeri casuali secondo distribuzioni

In [1]:
import ROOT as R
import plotly.express as px
import numpy as np
import scipy
import math
from tqdm import tqdm
import time
c=R.TCanvas()
%jsroot

Welcome to JupyROOT 6.16/00


### Generazione tramite metodo della trasformata -- integrale numerico

In questo primo codice la trasformata viene generata tramite calcolo del valore numerico dell'integrale 

In [2]:
def thetaeffe(x, par):
    toreturn=1/((par[0]*math.cos(x[0])**2.0) +math.sin(x[0])**2.0)
    return toreturn


In [3]:
def int_thetaeffe(x, par):
    func=R.TF1("thetaeffe",thetaeffe,0, R.TMath.Pi(),1)
    func.SetParameter(0,par[0])
    integral=func.Integral(0,x[0])

    return integral


In [4]:
def gen_transormation(alpha=0.5, points=100,seed=12, nbins=100):
    func=R.TF1("thetaeffe",thetaeffe,0, R.TMath.Pi(),1)
    func.SetParameter(0,alpha)
    start=time.time()
    func_T=R.TF1("int_thetaeffe",int_thetaeffe,0, R.TMath.Pi(),1)
    func_T.SetParameter(0,alpha)
    integ_tot=func.Integral(0,R.TMath.Pi())
    rand=R.TRandom3()
    h_generated = R.TH1F("h_generated","Generated numbers",nbins,0,2*R.TMath.Pi())
    for z in tqdm(range(0,points)):
        rand_num=rand.Rndm()
        if rand_num>0.5:
            C=R.TMath.Pi()
        else:
            C=0
        rand_num=rand.Rndm()
        h_generated.Fill((func_T.GetX(rand_num*integ_tot))+C)
    end=time.time()
    elapsed_time=end-start
    return h_generated,time,func

In [5]:
h_generated,time,func=gen_transormation(points=10,seed=0,nbins=50)
h_generated.Draw()
c.Draw()

100%|██████████| 10/10 [00:05<00:00,  1.64it/s]


Il  processo e lentissimo, con circa 7 secondi per punto (immagino che con l'integrazione numerica, root calcoli ogni volta la funzione su tutto il range prima  di trovare la X corrispondente).
Immmagino si possa tunare il binning dell'oggetto TF1, ma posso cercare metodo più furbi

In [6]:
func=R.TF1("thetaeffe",thetaeffe,0, 2*R.TMath.Pi(),1)
func.SetParameter(0,0.5)
h_scale=h_generated.Clone()
h_scale.Scale(func.Integral(0,R.TMath.Pi()*2)/(h_generated.Integral()*h_generated.GetBinWidth(2)))
h_scale.Draw("histo")
func.Draw("same")
c.Draw()

### Generazione tramite metodo della trasformata -- LUT



Prima di provate tramite versione analitica della trasformata (cosa che farò nell'esercizio sullo scattering compton), ho provato a ridurre il tempo di esecuzione calcolando l'integrale numerico della funzione una volta sola, salvando i valori in una lista

In [7]:
import ROOT as R
import plotly.express as px
import numpy as np
import scipy
import math
from tqdm import tqdm
import time
R.gStyle.SetOptStat(0)
c=R.TCanvas()
%jsroot

In [8]:
def thetaeffe(x, par):
    toreturn=1/((par[0]*math.cos(x[0])**2.0) +math.sin(x[0])**2.0)
    return toreturn

In [9]:
def int_thetaeffe(x, par):
    func=R.TF1("thetaeffe",thetaeffe,0, R.TMath.Pi(),1)
    func.SetParameter(0,par[0])
    integral=func.Integral(0,x[0])
    return integral

In [10]:
def buil_theateffe_int(par, bin_num):
    """
    Costruisco la tabella coi valori dell'integrale. Il numero di bin dovrebbe essere il DOPPIO del necessario (vista la definizione fra 0 e pi).
    Lo tengo così, per evitare problemi con nbin dispari
    """
    start=time.time()
    bins=bin_num
    func=R.TF1("thetaeffe",thetaeffe,0, R.TMath.Pi(),1)
    func.SetParameter(0,par)
    res_list=[]
    x_list=[]
    for x in tqdm(range (0,bins+1)):
        res_list.append(func.Integral(0,x*R.TMath.Pi()/(bins) +R.TMath.Pi()/bins/2 ))
        x_list.append(x*R.TMath.Pi()/(bins)+R.TMath.Pi()/bins/2)
    elap_time=time.time()-start
    print ("\nTime spent building look up table={}\n".format(elap_time))
    return (x_list,res_list)

In [11]:
def optimized_founder(sorted_list, trgt):
    """
    Sapendo di aver costruito una lista di valori ordinati crescenti, ho scritto un metodo ottimizzato per trovare la X corrispondente al valore dell'integrale,
    visto che non ho trovato metodi per cercare un valore traendo vantaggio dall'ordinamento
    """
    for num,elem in enumerate(sorted_list):
        if elem-trgt>0:
            if num!=0 and abs(sorted_list[num-1]-trgt)< abs(elem-trgt):
                return (num-1)
            else:
                return num
    return len (sorted_list)-1

In [12]:
def gen_transormation_look_up(alpha=0.5, points=1000000,seed=0, nbins=1000):
    return_list=[]
    func=R.TF1("thetaeffe",thetaeffe,0, R.TMath.Pi(),1)
    func.SetParameter(0,alpha)
    start=time.time()
    integ_tot=func.Integral(0,R.TMath.Pi())
    x_list,res_list=buil_theateffe_int(alpha,nbins)
    max_ = func.GetMaximum()

    if seed==0:
        rand=R.TRandom3(int(time.time()))
    else:
        rand=R.TRandom3(seed)
    h_generated = R.TH1F("h_generated","Generated numbers",nbins,0,2*R.TMath.Pi())
    for z in tqdm(range(0,points)):
        rand_num=rand.Rndm()
        if rand_num>0.5:
            C=R.TMath.Pi()
        else:
            C=0
        rand_num=rand.Rndm()
        index = optimized_founder(res_list,rand_num*integ_tot)
        return_list.append(x_list[index]+C)
        h_generated.Fill(x_list[index]+C)
    end=time.time()
    elapsed_time=end-start
    return h_generated,elapsed_time,func,return_list

In [13]:
h_generated_0,elap_time,func,return_list=gen_transormation_look_up(0.5,1000000,0,1000)


  0%|          | 0/1000000 [00:00<?, ?it/s]


Time spent building look up table=0.055599212646484375



100%|██████████| 1000000/1000000 [00:38<00:00, 26009.28it/s]
Warning in <TROOT::Append>: Replacing existing TH1: h_generated (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_generated (Potential memory leak).


In [14]:
h_generated_0.Scale(func.Integral(0,R.TMath.Pi()*2)/(h_generated_0.Integral()*h_generated_0.GetBinWidth(2)))
h_generated_0.Draw("histo")
h_generated_0.SetName("h_generated_0")
func.SetRange(0,R.TMath.Pi()*2)
func.Draw("same")
c.Draw()
print ("Elapsed_time = {}".format(elap_time))

Elapsed_time = 38.51023054122925


Questo metodo è molto più veloce del metodo con l'integrazione numerica. Ovviamente la velocità dipende dalnumero di bin richiesti. 

### Generazione tramite metodo acc/rej


In [15]:
def thetaeffe(x, par):
    toreturn=1/((par[0]*math.cos(x[0])**2.0) +math.sin(x[0])**2.0)
    return toreturn


In [16]:
def gen_acc_rej(alpha=0.5, points=1000000,seed=12, nbins=1000):
    func=R.TF1("thetaeffe",thetaeffe,0, R.TMath.Pi(),1)
    func.SetParameter(0,alpha)
    start=time.time()
    rand=R.TRandom3()
    max_ = func.GetMaximum()
    h_generated = R.TH1F("h_generated","Generated numbers",nbins,0,2*R.TMath.Pi())
    z=0
    with tqdm(total=points) as pbar:
        while z<points:
            rand_num=rand.Rndm()
            if rand_num>0.5:
                C=R.TMath.Pi()
            else:
                C=0
            r1=rand.Rndm()*R.TMath.Pi()
            r2=rand.Rndm()
            if r2<func.Eval(r1)/max_:
                h_generated.Fill(r1+C)
                z+=1
                pbar.update(1)
    end=time.time()
    elapsed_time=end-start
    return h_generated,elapsed_time,func

In [17]:
def root_function(alpha=0.5, points=1000000,seed=12, nbins=1000):
    func=R.TF1("thetaeffe",thetaeffe,0, 2*R.TMath.Pi(),1)
    func.SetParameter(0,alpha)
    start=time.time()
    rand=R.TRandom3()
    max_ = func.GetMaximum()
    h_generated = R.TH1F("h_generated","Generated numbers",nbins,0,2*R.TMath.Pi())
    z=0
    for z in tqdm(range(0,points)):
        h_generated.Fill(func.GetRandom())
    end=time.time()
    elapsed_time=end-start
    return h_generated,elapsed_time,func

In [18]:
h_generated_1,elap_time,func=gen_acc_rej()


100%|██████████| 1000000/1000000 [00:08<00:00, 116583.68it/s]


In [19]:
h_generated_1.Scale(func.Integral(0,R.TMath.Pi()*2)/(h_generated_1.Integral()*h_generated_1.GetBinWidth(2)))
h_generated_1.Draw("histo")
h_generated_1.SetName("h_generated_1")
func.SetRange(0,R.TMath.Pi()*2)
func.Draw("same")
c.Draw()
print ("Elapsed_time = {}".format(elap_time))

Elapsed_time = 8.580133199691772


Il metodo performa meglio di quello della trasformata con LUT (alemno quando il numero di bin richiesti è >100)

## Confronto fra i vari metodi

Genero i dati anche con la funzione disponibile nella libreria di ROOT.

In [20]:
h_generated_2,elap_time_2,func_2=root_function()


100%|██████████| 1000000/1000000 [00:01<00:00, 596080.11it/s]


In [21]:
h_generated_2.Scale(func_2.Integral(0,R.TMath.Pi()*2)/(h_generated_2.Integral()*h_generated_2.GetBinWidth(2)))
h_generated_2.Draw("histo")
h_generated_2.SetName("h_generated_2")
func_2.SetRange(0,R.TMath.Pi()*2)
func_2.Draw("same")
c.Draw()
print ("Elapsed_time = {}".format(elap_time_2))

Elapsed_time = 1.6794054508209229


Confrontando le distribuzuoni risultano tutte equivalenti

In [22]:
h_generated_2.SetLineColor(3)
h_generated_1.SetLineColor(4)
h_generated_0.SetLineColor(6)
h_generated_2.SetLineStyle(2)
h_generated_1.SetLineStyle(2)
h_generated_0.SetLineStyle(2)

h_generated_0.Draw("histo")
h_generated_1.Draw("histo same")

h_generated_2.Draw("histo same")
leg=R.TLegend(0.58,0.7,0.85,0.9);
leg.AddEntry("h_generated_2","Generated via root function","l")
leg.AddEntry("h_generated_1","Acc/rej method","l")
leg.AddEntry("h_generated_0","Transf-LUT method","l")

leg.Draw()
c.Draw()

### Compton scatter


Ho generato gli angoli con entrambi i metodo. Per il metodo delle trasformate, la funzione era facilmante integrabile analiticamente, quindi ho usato l'integrale calcolato.

In [23]:
import array

In [24]:
Me=0.5
C=(1/137)**2/(2*Me**2)
def scat_comp(x, par):
    toreturn=C*Me/(Me+par[0]*x[0])
    return toreturn

def trans_func(x, par):
    toreturn = (Me*math.log(par[0]*x[0]+Me))/par[0]*C
    return toreturn


Ho generalizzato le funzioni per la generzione al fine di prendere in input una qualsasi T1F di ROOT

In [25]:
def gen_acc_rej(func, points=1000000,seed=12, nbins=500):
    start=time.time()
    rand=R.TRandom3()
    max_ = func.GetMaximum()
    h_generated = R.TH1F("h_generated","Generated numbers",nbins,0,2)
    z=0
    with tqdm(total=points) as pbar:
        while z<points:          
            r1=rand.Rndm()*2
            r2=rand.Rndm()
            if r2<func.Eval(r1)/max_:
                h_generated.Fill(r1)
                z+=1
                pbar.update(1)
    end=time.time()
    elapsed_time=end-start
    return h_generated,elapsed_time,func

In [26]:
def gen_transormation(func, trans_func, points=1000000,seed=0, nbins=500):
    start=time.time()
    max_ = trans_func.GetMaximum()
    min_ = trans_func.GetMinimum()

    if seed==0:
        rand=R.TRandom3(int(time.time()))
    else:
        rand=R.TRandom3(seed)
    
    h_generated = R.TH1F("h_generated","Generated numbers",nbins,0,2)
    for z in tqdm(range(0,points)):
        rand_num=rand.Rndm()
        h_generated.Fill(trans_func.GetX(rand_num*(max_-min_)-abs(min_)))
    end=time.time()
    elapsed_time=end-start
    return h_generated,elapsed_time,func

In [27]:
func=R.TF1("scat_comp",scat_comp,0,2,1)
func.SetParameter(0,0.5) #K = 5 MeV
trans_func=R.TF1("trans_scat_comp",trans_func,0,2,1)
trans_func.SetParameter(0,0.5)
h_generated,elapsed_time,func=gen_acc_rej(func, points=10000)
h_generated_T,elapsed_time_T,func_T=gen_transormation(func,trans_func, points=10000)


100%|██████████| 10000/10000 [00:00<00:00, 12809.75it/s]
Warning in <TROOT::Append>: Replacing existing TH1: h_generated (Potential memory leak).


In [28]:
h_ori=h_generated.Clone()
h_ori_T=h_generated_T.Clone()

h_generated.Scale(func.Integral(0,2)/(h_generated.Integral()*h_generated.GetBinWidth(2)))
h_generated_T.Scale(func.Integral(0,2)/(h_generated_T.Integral()*h_generated_T.GetBinWidth(2)))
h_generated_T.SetLineColor(2)
h_generated.Draw("histo")
h_generated_T.Draw("histo same")

func.SetRange(0,2)
func.Draw("same")
c.Draw()
print ("Elapsed_time acc/rej = {}".format(elapsed_time))
print ("Elapsed_time transf= {}".format(elapsed_time_T))

Elapsed_time acc/rej = 0.06419253349304199
Elapsed_time transf= 0.7862064838409424


Per completare il lavoro, converto nuovamente gli angoli

In [29]:
def convert_to_angle(histo):
    zeros=np.zeros((histo.GetNbinsX())+1)
    xbins = array.array('d', zeros)
    for bin_num in range(1, histo.GetNbinsX()+1):
        ext_1=math.acos(1-histo.GetBinLowEdge(bin_num))
        xbins[bin_num]=ext_1
    new_hist=R.TH1F("h_angle","Generated angle",histo.GetNbinsX(),xbins)
    for bin_num in range(1, histo.GetNbinsX()):
        for t in range (0,int(histo.GetBinContent(bin_num))):
            new_hist.Fill(math.acos(1-histo.GetBinCenter(bin_num)))
    return new_hist

In [30]:
h_ori.GetBinContent(12)


33.0

In [31]:
histo_angle=convert_to_angle(h_ori)
histo_angle_T=convert_to_angle(h_ori_T)

Warning in <TROOT::Append>: Replacing existing TH1: h_angle (Potential memory leak).


In [32]:
histo_angle.Draw()
histo_angle_T.SetLineColor(2)
histo_angle_T.Draw("same")
c.Draw()